In [1]:
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, accuracy_score


In [2]:
def pad_features(arr, target_cols):
    """Pads the feature array with zeros to match the target number of columns."""
    if arr.shape[1] < target_cols:
        pad_width = target_cols - arr.shape[1]
        arr = np.pad(arr, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    return arr

def normalize_features(X):
    scaler = StandardScaler()
    return scaler.fit_transform(X), scaler

def train_svm_with_grid_search(X_train, y_train):
    param_grid = {
        'C': [30],
        'gamma': [7e-2, 7e-3, 7e-4, 'scale', 'auto'],
        'kernel': ['rbf'],
    }
    grid_search = GridSearchCV(svm.SVC(probability=True), param_grid, cv=2, n_jobs=-1, verbose=3)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

def compute_eer(y_true, y_scores):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fnr - fpr))]
    eer = fpr[np.nanargmin(np.abs(fnr - fpr))]
    return eer, eer_threshold


In [3]:
#Without filteration

ram_train_data = np.load('./Without_Filteration/ram_train.npy')
ram_test_data = np.load('./Without_Filteration/ram_test.npy')
imposter_train_data = np.load('./Without_Filteration/train_data.npy')
imposter_test_data = np.load('./Without_Filteration/test_data.npy')

# Align feature dimensions
max_feature_size = max(
    ram_train_data.shape[1], imposter_train_data.shape[1], 
    ram_test_data.shape[1], imposter_test_data.shape[1]
)

ram_train_data = pad_features(ram_train_data, max_feature_size)
imposter_train_data = pad_features(imposter_train_data, max_feature_size)
ram_test_data = pad_features(ram_test_data, max_feature_size)
imposter_test_data = pad_features(imposter_test_data, max_feature_size)

# Prepare training data
X_train = np.vstack([ram_train_data, imposter_train_data])
y_train = [1] * len(ram_train_data) + [0] * len(imposter_train_data)

# Prepare test data
X_test = np.vstack([ram_test_data, imposter_test_data])
y_test = [1] * len(ram_test_data) + [0] * len(imposter_test_data)

# Normalize features
X_train_scaled, scaler = normalize_features(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM
clf = train_svm_with_grid_search(X_train_scaled, y_train)

# Get decision scores
y_scores = clf.predict_proba(X_test_scaled)[:, 1]

print("Data without filtration:")
# Compute EER
eer, eer_threshold = compute_eer(y_test, y_scores)
print(f"EER: {eer * 100:.2f}% at threshold: {eer_threshold}")

# Evaluate accuracy
y_pred = (y_scores >= eer_threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy at EER threshold: {accuracy * 100:.2f}%")

Fitting 2 folds for each of 5 candidates, totalling 10 fits


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Data without filtration:
EER: 38.46% at threshold: 0.5578290988907009
Accuracy at EER threshold: 61.54%


In [4]:
#With Wiener filteration

ram_train_data = np.load('./With_Wiener_Filteration/ram_train.npy')
ram_test_data = np.load('./With_Wiener_Filteration/ram_test.npy')
imposter_train_data = np.load('./With_Wiener_Filteration/train_data.npy')
imposter_test_data = np.load('./With_Wiener_Filteration/test_data.npy')

# Align feature dimensions
max_feature_size = max(
    ram_train_data.shape[1], imposter_train_data.shape[1], 
    ram_test_data.shape[1], imposter_test_data.shape[1]
)

ram_train_data = pad_features(ram_train_data, max_feature_size)
imposter_train_data = pad_features(imposter_train_data, max_feature_size)
ram_test_data = pad_features(ram_test_data, max_feature_size)
imposter_test_data = pad_features(imposter_test_data, max_feature_size)

# Prepare training data
X_train = np.vstack([ram_train_data, imposter_train_data])
y_train = [1] * len(ram_train_data) + [0] * len(imposter_train_data)

# Prepare test data
X_test = np.vstack([ram_test_data, imposter_test_data])
y_test = [1] * len(ram_test_data) + [0] * len(imposter_test_data)

# Normalize features
X_train_scaled, scaler = normalize_features(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM
clf = train_svm_with_grid_search(X_train_scaled, y_train)

# Get decision scores
y_scores = clf.predict_proba(X_test_scaled)[:, 1]

print("Data With Wiener filteration:")
# Compute EER
eer, eer_threshold = compute_eer(y_test, y_scores)
print(f"EER: {eer * 100:.2f}% at threshold: {eer_threshold}")

# Evaluate accuracy
y_pred = (y_scores >= eer_threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy at EER threshold: {accuracy * 100:.2f}%")

Fitting 2 folds for each of 5 candidates, totalling 10 fits
Data With Wiener filteration:
EER: 38.46% at threshold: 0.8776670121024768
Accuracy at EER threshold: 61.54%
